<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [1]:
import requests
import pandas as pd
import numpy as np
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## [Day 20](https://adventofcode.com/2021/day/20): Trench Map
### Part I
- **Unknown**: The count of lit pixels in the image after two enhancements.
- **Data**: The image enhancement algorithm and an input image.
- **Condition**:
  - The input image is a two-dimensional grid of light and dark pixels.
  - The 9-bit binary number resulting from transformation of 3x3 pixels around the given pixel gives an index of the output pixel.
  - The images are infinite in size surrounded by dark pixels.
- **Plan**:
  - Parse the input to get the image enhancement string `enstr` and the image `img`.
  - Add the boudary to simplify the filtering algorithm.
  - Define the filtering function.

In [2]:
example = """..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###"""

In [37]:
def parse(input):
  data = input.split('\n')
  enstr = data[0].replace('.','0').replace('#','1')
  img = np.array([list(row.replace('.','0').replace('#','1')) for row in data[2:]]).astype(int)
  return enstr, img

enstr, img = parse(example)
print(enstr)
print(img)
print(img[0][0], img[-1][-1])

00101001111101010101110110000011101101001110111100111110010000100100110011100111111011100011110010011111001100101111100011010100101100101000000101110111111011101111000101101100100100111110000010100001110010110000001000001001001001100100011011111101111011110101000100000001001010100011110110100000010010001101011001000110101100111010000001010000000101010111101110110001000001111010010010110100001100101111000011000110010001000000101000000010000000110011110010001010100011001010011100111110000000010011110000001001
[[1 0 0 1 0]
 [1 0 0 0 0]
 [1 1 0 0 1]
 [0 0 1 0 0]
 [0 0 1 1 1]]
1 1


### Using `.` and `#`
Let's consider leaving the pixel representaion as characters `.` and `#`. It is easy to index the image enhancement string `enstr`. When transforming neighborhood of a pixel into an index, the string <-> number trasformation will take place anyway.

But the adding boundary and other transformation of a numpy array is much simpler when working with integers.

### Using `0` and `1`
Trasforming `.` -> `0`/0 and `#` -> `1`/1 ensures cleaner numpy coding. For reading output values from the image enhancement string is still clearer to work with string of zeros and ones.

### Add Boundary
The next step is adding a dark pixel boundary to infer neighborhood easily. This algorithmic pattern is used in [Day 9](AoC_08.ipynb) - Smoke Basin.

In [19]:
def add_boundary(img):
  """Add zeros to the boundary of an image."""
  # Add vertical boundaries
  b = np.zeros(img.shape[1]).reshape(1,-1)
  out = np.vstack((b,img,b))
  # Add horizontal boudnaries
  b = np.zeros(out.shape[0]).reshape(-1,1)
  out = np.hstack((b,out,b))
  return out.astype(int)
add_boundary(img)

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0]])

### Filter Image
Filtering means applying the image enhancement alorithm to the image.

In [43]:
def get_output_pixel(img, enstr, r, c):
  """Return the value of output pixel"""
  # Determine the binary value of the neigborhood
  m = img[r-1:r+2,c-1:c+2].flatten()
  out = '0b' + ''.join([str(n) for n in m])
  # Convert the binary number and find the output pixel
  out = enstr[int(out,2)]
  return int(out)

def filter_image(img, enstr):
  """Add a boundary, apply the filter, return the filtered image"""
  # Add boundary twice to the input image
  imgb = add_boundary(add_boundary(img))
  # Initialize the output image
  out = np.zeros(imgb.shape)
  for r in range(1, out.shape[0]-1):
    for c in range(1, out.shape[1]-1):
      out[r,c] = get_output_pixel(imgb, enstr, r, c)
  return out.astype(int)

img_twice = filter_image(filter_image(img, enstr), enstr)
img_twice

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### Solve the part I

In [47]:
def solve_part1(input):
  # Parse input
  enstr, img = parse(input)
  # Count lit pixels after two enhancements
  img_twice = filter_image(filter_image(img, enstr), enstr)
  return img_twice[img_twice>0].size
solve_part1(example)

35

In [51]:
r = requests.get(path+'AoC2021_20.txt')
enstr, img = parse(r.text[:-1])
img[-3:,-3:]

array([[0, 0, 1],
       [1, 1, 1],
       [1, 0, 1]])

In [50]:
solve_part1(r.text[:-1])

6124

'6124` : That's not the right answer; your answer is too high.

### Part II
- **Unknown**: 
- **Data**: 
- **Condition**:
  - 
  - 
- **Plan**:
  - 
